<a href="https://colab.research.google.com/github/AlejandroMllo/UAV-Human-Influence-Detection/blob/master/NN_%2B_Line_Profiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import cmath
import math

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import normalize

%matplotlib inline

In [0]:
def laguerre_gauss_filter(side_size, w):
  """
  Computes and returns a matrix containing the Laguerre-Gauss filter.
  """
  
  scale = (1.j * math.pow(math.pi, 2) * math.pow(w, 4))
  # Not sure if `scale` is the appropriate name
  power_scale = -math.pow(math.pi, 2) * math.pow(w, 2)
  # Not sure if `power_scale` is the appropriate name
  
  filter = np.zeros((side_size, side_size), dtype=complex)
  
  for x in range(side_size):
    x_squared = math.pow(x, 2)
    for y in range(side_size):
      power = cmath.exp(power_scale * (x_squared + math.pow(y, 2)))
      
      filter[x, y] = scale * complex(x, y) * power
    # print('Coord:', x, y, scale * complex(x, y) * power)
      
  return filter

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def rgb2grayscale(rgb_img):
  
  if rgb_img.ndim != 3:
    # print('returning non rgb image')
    return rgb_img
  
  img = np.zeros(rgb_img.shape)
  img[:, :, 0] = rgb_img[:, :, 0] * 0.2125 # RED
  img[:, :, 1] = rgb_img[:, :, 1] * 0.7154 # GREEN
  img[:, :, 2] = rgb_img[:, :, 2] * 0.0721 # BLUE

  return np.sum(img, axis=2)

In [0]:
def kernel_transform(batch, kernel=np.fft.fft2):
  
  transformed_batch = []
  
  for instance in batch:
    
    instance = rgb2grayscale(instance)
    transformed = kernel(instance)
    transformed_batch.append(transformed)
    
  return np.array(transformed_batch)


def convolve(transformed_kernel, batch):
  
  convolved_batch = []
  
  for instance in batch:
    conv = np.multiply(transformed_kernel, instance)
    convolved_batch.append(conv)
    
  return np.array(convolved_batch)


def shift(batch):
  
  shifted_batch = []
  
  for instance in batch:
    shifted = np.fft.fftshift(instance)
    shifted_batch.append(shifted)
    
  return np.array(shifted_batch)


def line_profile(batch, axis=1):
  # Axis 0: y axis
  # Axis 1: x axis
  # Equivalent axes definition to numpy.
  
  line_profiles = []
  
  num_instances = batch.shape[0]
  
  for i in range(num_instances):
   
    instance = batch[i]
    axis_length = instance.shape[1 - axis] - 1
    
    if axis == 0:
      line_profiles.append(instance[:,axis_length//2])
    else:
      line_profiles.append(instance[axis_length//2,:])
      
  return np.array(line_profiles)

In [0]:
def ft_pipeline(transformed_filter, sample):
  
  transformed = kernel_transform(sample)
  convolved = convolve(transformed_filter, transformed)
  shifted = shift(convolved)
  x_profile = line_profile(shifted, axis=1)
  y_profile = line_profile(shifted, axis=0)
  
  return x_profile, y_profile

In [0]:
vectorized_phase = np.vectorize(cmath.phase)
vectorized_amplitude = np.vectorize(np.abs)

### Data Handling

In [0]:
# -- Find files

def find_files(path):
  
  files = next(os.walk(path))[2]
  return np.array(sorted(
            files, key=lambda f: int("".join(list(filter(str.isdigit, f))))
         ))
  

def load_images(image_names, path):
  
  images = []
  
  for name in image_names:
    img_path = path + name
    images.append(mpimg.imread(img_path))
    
  return images

# Architecture Search

## Shapes

In [0]:
lg_28 = laguerre_gauss_filter(28, 0.9)
ft_lg_28 = np.fft.fft2(lg_28)

### Hand-drawn unfilled shapes

In [0]:
shapes_dataset_path = 'drive/My Drive/PI 1/Datasets/shapes'

triangles_path = shapes_dataset_path + '/triangles/'
squares_path = shapes_dataset_path + '/squares/'
circles_path = shapes_dataset_path + '/circles/'


triangles = find_files(triangles_path)
squares = find_files(squares_path)
circles = find_files(circles_path)


triangle_images = load_images(triangles, triangles_path)
square_images = load_images(squares, squares_path)
circle_images = load_images(circles, circles_path)

In [11]:
shapes_classes = [0, 1, 2] # 0=triangles; 1=squares; 2=circles
shapes_images = np.array([triangle_images, square_images, circle_images]).reshape((-1, 28, 28, 3))
shapes_labels = np.array( ([0]*len(triangle_images)) + ([1]*len(square_images)) + ([2]*len(circle_images)) )

print(shapes_images.shape)

(300, 28, 28, 3)


In [0]:
from sklearn.model_selection import train_test_split

# Extract features from images
x_profiles, y_profiles = ft_pipeline(ft_lg_28, shapes_images)

x_profiles = vectorized_amplitude(np.concatenate((x_profiles, y_profiles), axis=1))

# Split datasets
train_features, val_features, train_labels, val_labels = \
  train_test_split(x_profiles, shapes_labels, test_size=0.6, random_state=420)

validation_features, test_features, validation_labels, test_labels = \
  train_test_split(val_features, val_labels, test_size=0.5, random_state=0)

#### KNN Classifier

In [40]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(4)
clf.fit(train_features, train_labels)

score = clf.score(validation_features, validation_labels)

print('KNN score =', score)

KNN score = 0.8222222222222222


In [41]:
clf.score(test_features, test_labels)

0.7777777777777778

#### MLP

In [42]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

num_classes = 3

# convert class vectors to binary class matrices
train_labels = keras.utils.to_categorical(train_labels, num_classes)
validation_labels = keras.utils.to_categorical(validation_labels, num_classes)

Using TensorFlow backend.


In [43]:
model = Sequential()
model.add(Dense(28, activation='relu', input_dim=56, name='my1'))
model.add(Dropout(0.25))
model.add(Dense(56, activation='relu', name='my3'))
model.add(Dropout(0.25))
model.add(Dense(56, activation='relu', name='my4'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu', name='my5'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_hinge, #categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

epochs = 2048
batch_size = 8

model.fit(train_features, train_labels,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(validation_features, validation_labels))
score = model.evaluate(validation_features, validation_labels, verbose=0)
print('Test loss (x_profiles):', score[0])
print('Test accuracy (x_profiles):', score[1])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 120 samples, validate on 90 samples
Epoch 1/2048
120/120 [==============================] - 1s 8ms/step - loss: 1.0045 - acc: 0.3083 - val_loss: 1.0005 - val_acc: 0.4222
Epoch 2/2048
120/120 [==============================] - 0s 326us/step - loss: 1.0013 - acc: 0.3583 - val_loss: 1.0008 - val_acc: 0.3889
Epoch 3/2048
120/120 [==============================] - 0s 307us/step - loss: 1.0013 - acc: 0.3250 - val_loss: 1.0006 - val_acc: 0.3889
Epoch 4/2048
120/120 [==============================] - 0s 301us/step - loss: 1.0012 - acc: 0.3000 - val_loss: 1.0002 - val_acc: 0.3556
Epoch 5/2048
120/120 [==============================] - 0s 309us/step - loss: 1.0005 - acc: 0.3083 - val_loss: 1.0002 - val_acc: 0.3889
Epoch 6/2048
120/120 [==============================] - 0s 304

### Geometric Shapes

In [0]:
shapes_dataset_path = 'drive/My Drive/PI 1/Datasets/shapes'

triangles_path = shapes_dataset_path + '/triangles/'
squares_path = shapes_dataset_path + '/squares/'
circles_path = shapes_dataset_path + '/circles/'


triangles = find_files(triangles_path)
squares = find_files(squares_path)
circles = find_files(circles_path)


triangle_images = load_images(triangles, triangles_path)
square_images = load_images(squares, squares_path)
circle_images = load_images(circles, circles_path)